# Funciones para encontrar usuarios a testear

In [1]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from tqdm import tqdm
import pickle

In [2]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf.loc[:,'IP Address'].values[0]
}

In [3]:
# targetUsers = pd.DataFrame(columns=['username'])

targetUsers = pd.read_pickle('../datasets/targetUsersDataset2.pkl').head(1)
print(len(targetUsers))
targetUsers.head()

1


,username
235491971,gsrcaballero


In [4]:
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getNewPage(term, searchType, maxPosition):
    if(searchType != 'q'):
        term = '#'+term
    pageResponse = requests.get(
        'https://twitter.com/i/search/timeline',
        params={
            'vertical': 'news',
            'q': term,
            'max_position': maxPosition,
            'include_entities': 1,
            'include_available_features': 1,
            'reset_error_state': False
        },
        proxies=proxies,
        headers=headersMore
    )
    return(pageResponse.json())
#     print(pageResponse.json().keys())
#     print(pageResponse.json()['min_position'])
#     display(BeautifulSoup(pageResponse.json()['items_html'], 'html.parser'))
    return pageResponse.json()['min_position'], BeautifulSoup(pageResponse.json()['items_html'], 'html.parser')

def getUserTweetData(tweet):
    try:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userId = tweet.findAll('div', attrs={'class':'js-stream-tweet'})[0].attrs['data-user-id']
        targetUsers.loc[userId] = [username]
    except:
        print('ERROR---------------------')
        print(tweet)
    return [username, userId]

def searchTerm(term, searchType='q', maxPages=15):
    pageResponse = False
    if(searchType == 'q'):
        pageResponse = requests.get('https://twitter.com/search?q='+term, headers=headersChrome, proxies=proxies)
    else:
        pageResponse = requests.get('https://twitter.com/hashtag/'+term, headers=headersChrome, proxies=proxies)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)
    for tweet in tweetList:
        getUserTweetData(tweet)
    nextKey = dataMax
    for pageNum in tqdm(range(maxPages)):
        response = getNewPage(term, searchType, nextKey)
        nextKey = response['min_position']
        htmlResponse = BeautifulSoup(response['items_html'], 'html.parser')
        if(len(htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)) == 0):
            print(response)
            print('NO MORE',htmlResponse)
            break
        for tweet in htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False):
            getUserTweetData(tweet)

# searchTerm('cfk', '#', 15)
# searchTerm('cambiemos', '#', 5)
# searchTerm('MacriVendepatria', '#', 5)
# searchTerm('JuntosPorElCambio', '#', 5)
# searchTerm('carrio', 'q', 100)

In [5]:
import threading
lock = threading.RLock()
def process_id(id):
    """process a single ID"""
    # fetch the data
    r = requests.get(url_t % id)
    # parse the JSON reply
    data = r.json()
    # and update some data with PUT
    requests.put(url_t % id, data=data)
    return data

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = searchTerm(id, 'q', 600)
    return store

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        with lock:
            t = Thread(target=process_range, args=(ids,store))
            threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    print('Done')
    print(len(targetUsers))
    display(targetUsers.head())
    return 'Done'

termsToSearch = [
    '%23defensoresdelcambio',
    '%23JuntosPorElCambio',
    '%23defendercambio',
    '%23NVM',
    '%23QueHizoAxelEnPBA',
    '%23MM2019',
    '%23YoEstoyConElCambio',
    '%23MacriPichetto2019',
    '%23Vidal2019', 
    '%23#VidalTeSigoEligiendo'
    '%23AlbertoMeMandoACocinar',
    '%23ElKirchnerismoEsOdio',
    '%23NoVuelvenMasEnSuDelictivaVida',
    '%23NoSomosBoludos',
    '%23GraboisLadronDePobres'
]

threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)
    
# termsToSearch = [
#     'macri FMI',
#     'sinceramente falso',
#     'ella miente',
#     'massa',
#     'cristina pobreza'
# ]
# threaded_process_range(8, termsToSearch)
# filename = 'targetUsersDataset.pkl'
# with open(filename, 'wb') as twUsersTarget:
#     pickle.dump(targetUsers, twUsersTarget)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-1f4c22c9e59f>", line 18, in process_range
    store[id] = searchTerm(id, 'q', 600)
  File "<ipython-input-4-1a8ef9aaff52>", line 50, in searchTerm
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
IndexError: list index out of range

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/600 [00:00<?, ?it/s]

ERROR---------------------
<li class="js-stream-item stream-item stream-item" data-item-id="1149875350798774272" data-item-type="tweet" id="stream-item-tweet-1149875350798774272">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet" data-component-context="tweet" data-conversation-id="1149368045032398848" data-disclosure-type="" data-follows-you="false" data-has-parent-tweet="true" data-is-reply-to="true" data-item-id="1149875350798774272" data-mentions="joseluisgioja" data-name="bastadefarsas" data-permalink-path="/bastadefarsas/status/1149875350798774272" data-reply-to-users-json='[{"id_str":"1149411512236281859","screen_name":"bastadefarsas","name":"bastadefarsas","emojified_name":{"text":"bastadefarsas","emojified_text_as_html":"bastadefarsas"}},{"id_str":"19722366","screen_name":"joseluisgioja","name":"Jos\u00e9 Luis Gioja","emojified_name":{"text":"Jos\u00e9 Luis Gioja","emojified_text_as_html":"Jo




  0%|          | 0/600 [00:00<?, ?it/s]



  0%|          | 0/600 [00:00<?, ?it/s]




  0%|          | 1/600 [00:01<16:42,  1.67s/it]A





  0%|          | 2/600 [00:02<14:25,  1.45s/it]


  0%|          | 1/600 [00:01<16:36,  1.66s/it]

ERROR---------------------
<li class="js-stream-item stream-item stream-item" data-item-id="1147681305892667392" data-item-type="tweet" id="stream-item-tweet-1147681305892667392">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content" data-component-context="tweet" data-conversation-id="1147681305892667392" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1147681305892667392" data-mentions="mauriciomacri" data-name="cristian" data-permalink-path="/crischeli/status/1147681305892667392" data-possibly-sensitive="true" data-reply-to-users-json='[{"id_str":"1396099964","screen_name":"crischeli","name":"cristian","emojified_name":{"text":"cristian","emojified_text_as_html":"cristian"}},{"id_str":"24900072","screen_name":"mauriciomacri","name":"Mauricio Macri","emojified_name":{"text":"Mauricio Macri","emojified_text_as_html":"Mauricio Macri"}}]' data-scre




  0%|          | 2/600 [00:03<15:48,  1.59s/it]




  0%|          | 1/600 [00:03<32:48,  3.29s/it]



  0%|          | 1/600 [00:03<33:20,  3.34s/it]

  0%|          | 2/600 [00:03<17:10,  1.72s/it]


  0%|          | 2/600 [00:04<26:53,  2.70s/it]

  0%|          | 3/600 [00:04<15:43,  1.58s/it]



  0%|          | 2/600 [00:05<29:12,  2.93s/it]


  0%|          | 3/600 [00:05<19:45,  1.99s/it]




  0%|          | 3/600 [00:07<25:07,  2.53s/it]

  1%|          | 4/600 [00:06<16:33,  1.67s/it]


  1%|          | 4/600 [00:07<18:32,  1.87s/it]



  1%|          | 5/600 [00:08<15:50,  1.60s/it]

  1%|          | 4/600 [00:09<23:16,  2.34s/it]


  1%|          | 5/600 [00:08<17:42,  1.79s/it]




  0%|          | 3/600 [00:08<29:54,  3.01s/it]



  1%|          | 5/600 [00:10<21:07,  2.13s/it]

  1%|          | 6/600 [00:10<17:26,  1.76s/it]

...

{'min_position': 'thGAVUV0VFVBaCwLyVoJ_56B8WgMC0peuT6_QfEhiMAxJjwusAAAH0P4BiTdLxqfwAAAAvD-3mox1XkAAP6U31zVdgAA_vvuAy16AFD-mFqXOXsAAP6RXNtNfgAA_qxWy41sAAD-qzgsEXUAAP7reAKpfgAQ_1LkXyFpABD-9eAjRWwAAP71ZjdFeQAw_uUZS0VsAOD-8W2FGX4AAP7ru0kVdgAA_ut2FU16AAD-61riAXsAAP6k3PK1bgAA_zGHuaFrABD--9cK7XUAEP738iytbgAA_vccDoF7AAD_PY-QcU8AAP9NZPWlaQAA_uuGlgV7ACD-0qk8eW4AAP70rQAVdgBg_ve8_V1sAAD-6BNV-X4AEP9MWZfpbAAQ_vWALnV-AAD_ER9e1XUAMP888FYxQgBg_ro4E8V5AAD-9kvfaX4AUP732FL5bgAg_vXp24lsABD_NEMFQWwAEP753S-taQAQ_vSsqZ1rAAD-lNC9aXUAcP9G83r9TQAg_0nwtSl7AGD_SEk0EXYAEP7rMdWBawAQ_u3-8vFuADD-68f3kXYAAP7W2qC5aQABUCFQAVECUAVQAVAAA=', 'has_more_items': False, 'items_html': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n', 'new_latent_count': 0, 'focused_refresh_interval': 240000}
NO MORE 











  1%|▏         | 8/600 [00:09<11:13,  1.14s/it]







  2%|▏         | 9/600 [00:10<10:55,  1.11s/it]







  2%|▏         | 10/600 [00:11<10:22,  1.06s/it]

{'min_position': 'thGAVUV0VFVBaCwLqZ5fj06B8WgoC22cW57_QfEhjcAxJjwusAAAH0P4BiTdLxqfwAAAA5D_NwTDIWkAAP7zomTJeQAg_pB3I1FsAAD-oK7_BXoAAP9SNO-degAA_pbMXjFsABD-o8rlVX4AEP9I0uL1egAA_xHAQIF5AFD_AVKM0XoAIP8oOsdBewBQ_1B6I712AAD_ILOZMXUAAP9R90V5dgAQ_o598zVsABD_OKj3hXsAEP6N4eOBbAAA_zB1UZ16ADD-qiIobX4AEP9TYyhhaQAA_qDRE6V5ACD-l48KgXUAQP6VKaupewAA_pCblfFpACD_OlqMWXYAAP8nYmFJbgAQ_xQFYjF6AAD_MjL2yXsAIP8B2B0daQAQ_0prtblMABD_TspgyWsAMP6jRhERawAg_xXC2jV1AAD_KU7q9W4AAP9K9FehdgAA_uo-WiF5ABD-lUTjcWkAIP6lw7UNbgAA_0sV0iFpAAD_UlNYTXUAAP65OFmJbAAA_uCNKOF-AAD-o1s4vXoAEP6YeU-JbgAA_0WPsfVCAAD-j3CZ4WkAAP6Z_ylRbgAA_xA29f1pAAD-mZV80XsAEP6iSL69fgBg_wmsBRVpABD-ofSsPXoAAP6jKNwhbAAA_07gaLV6ABD_TelelW4AAP9Np4u9fgAA_yhvvalrABFQIVABUUJQBVABUAAA==', 'has_more_items': False, 'items_html': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n', 'new_latent_count': 0, 'focused_refresh_interval': 30000}
NO MORE 

Done
685


,username
235491971,gsrcaballero
171523709,EleonoraRiera
720730731492810752,yoconelcambio
816861181650550786,marcela_mym
823946465932361729,Santamehijos


'Done'

In [6]:
len(targetUsers)

685

In [7]:
#guardar resultados
import pickle
filename = '../datasets/targetUsersDataset_m.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)